<a href="https://colab.research.google.com/github/ydicsey/ipynb_code/blob/main/myHMM_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENV SetUp

In [ ]:
!pip install hmmlearn
import pandas as pd
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# myHMM implementation

In [ ]:
# avoid mul to 0

def LogAdd(*numbers):
    tmp = 0
    for n in numbers:
        if n == 0:
            continue;
        tmp += np.log(n)
    return (np.exp(tmp))

In [ ]:
import numpy as np

class HMM:
    """
    arrtibutes
    |               myHMM                   |  hmmlearn
    A(n*n) : transition probability matrix  => transmat_
    B(n*m) : emission probability matrix    => emissionprob_
    pi(1*n): Initial probablity vector      => startprob_
    N : number of hidden states             => n_states
    M : number of observable states         => n_features
    """
    def __init__(self, A, B, pi):
        self.A = np.array(A)
        self.B = np.array(B)
        self.pi = np.array(pi)
        self.N = self.A.shape[0]
        self.M = self.B.shape[1]

    def Forward(self,T,O):
        self.alpha = np.zeros((T,self.N),float)
        for i in range(self.N):
            self.alpha[0,i] = LogAdd(self.pi[i], self.B[i,O[0]])
    
        for t in range(T-1):
            for j in range(self.N):
                sum = 0.0
                for i in range(self.N):
                    sum += LogAdd(self.alpha[t,i], self.A[i,j])
                self.alpha[t+1,j] = LogAdd(sum, self.B[j,O[t+1]])

        self.fwp = 0.0
        for i in range(self.N):
            self.fwp += self.alpha[T-1,i]
        return self.fwp

    def Backward(self,T,O):
        self.beta = np.zeros((T,self.N),float)
        for i in range(self.N):
            self.beta[T-1,i] = 1.0
        for t in range(T-2,-1,-1):
            for i in range(self.N):
                sum = 0.0
                for j in range(self.N):
                    sum += LogAdd(self.A[i,j], self.B[j,O[t+1]], self.beta[t+1,j])
                self.beta[t,i] = sum
                
        self.bwp = 0.0
        for i in range(self.N):
            self.bwp += LogAdd(self.pi[i], self.B[i,O[0]], self.beta[0,i])
    
    def viterbi(self,O):
        T = len(O)
        # _init_
        delta = np.zeros((T,self.N), float)  
        phi = np.zeros((T,self.N), float)  
        I = np.zeros(T)
        for i in range(self.N):  
            delta[0,i] = LogAdd(self.pi[i], self.B[i,O[0]] )
            phi[0,i] = 0

        for t in range(1,T):  
            for i in range(self.N):                                  
                delta[t,i] = self.B[i,O[t]]*np.array([delta[t-1,j]*self.A[j,i]  for j in range(self.N)]).max()
                phi[t,i] = np.array([delta[t-1,j]*self.A[j,i]  for j in range(self.N)]).argmax()

        prob = delta[T-1,:].max()  
        I[T-1] = delta[T-1,:].argmax()
        # get state path  
        for t in range(T-2,-1,-1): 
            I[t] = phi[t+1,I[t+1]]
        return I, prob
        
    def ComputeGamma(self, T):
        self.gamma = np.zeros((T,self.N),float)
        for t in range(T):
            denominator = 0.0
            for j in range(self.N):
                self.gamma[t,j] = LogAdd(self.alpha[t,j], self.beta[t,j])
                denominator += self.gamma[t,j]
            for i in range(self.N):
                self.gamma[t,i] = LogAdd(self.gamma[t,i], 1/denominator)

    def ComputeXi(self,T,O):
        self.xi = np.zeros((T,self.N,self.N))
        for t in range(T-1):
            sum = 0.0
            for i in range(self.N):
                for j in range(self.N):
                    self.xi[t,i,j] = LogAdd(self.alpha[t,i], self.beta[t+1,j], self.A[i,j], self.B[j,O[t+1]])
                    sum += self.xi[t,i,j]
            for i in range(self.N):
                for j in range(self.N):
                    self.xi[t,i,j]  = LogAdd(self.xi[t,i,j] ,1/sum)
                    
    # Baum-Welch
    # input (obsversationS) consist of L sequences
    def BaumWelch(self, L, T, O, n_iter = 49):
        round = 0
        
        pi = np.zeros((T), float)
        denominatorA = np.zeros((self.N), float)
        denominatorB = np.zeros((self.N), float)
        numeratorA = np.zeros((self.N,self.N), float)
        numeratorB = np.zeros((self.N,self.M), float)
        
        while True :
            for l in range(L):
                leng = len(O[l])
                self.Forward(leng,O[l])
                self.Backward(leng,O[l])
                self.ComputeGamma(leng)
                self.ComputeXi(leng,O[l])
                for i in range(self.N):
                    pi[i] += self.gamma[0,i]
                    for t in range(leng-1): 
                        denominatorA[i] += self.gamma[t,i]
                        denominatorB[i] += self.gamma[t,i]
                    denominatorB[i] += self.gamma[leng-1,i]
                    
                    for j in range(self.N):
                        for t in range(leng-1):
                            numeratorA[i,j] += self.xi[t,i,j]
                    for k in range(self.M):
                        for t in range(leng):
                            if O[l][t] == k:
                                numeratorB[i,k] += self.gamma[t,i]
                            
            # M - step
            for i in range(self.N):
                self.pi[i] = pi[i]/L
                for j in range(self.N):
                    self.A[i,j] = LogAdd(numeratorA[i,j], 1/denominatorA[i])
                    numeratorA[i,j] = 0.0
                
                for k in range(self.M):
                    self.B[i,k] = LogAdd(numeratorB[i,k], 1/denominatorB[i])
                    numeratorB[i,k] = 0.0
                
                pi[i]=denominatorA[i]=denominatorB[i]=0.0;

            round += 1
            
            if round > n_iter :
                print ("iteration ",round)
                break
    def printhmm(self):
        print ("==================================================")
        print ("HMM content: n_states(N) =",self.N,",n_features(M) =",self.M)
        print ("\nhmm.pi",self.pi)
        for i in range(self.N):
            if i==0:
                print ("hmm.A ",self.A[i,:],"\t hmm.B ",self.B[i,:])
            else:
                print ("      ",self.A[i,:],"\t       ",self.B[i,:])
        print ("==================================================")

# train set 1

In [ ]:
A = [[0.34, 0.33, 0.33], [0.33, 0.34, 0.33], [0.33, 0.33, 0.34]]
B = [[0.34, 0.33, 0.33], [0.33, 0.34, 0.33], [0.33, 0.33, 0.34]]
pi = [0.33, 0.33, 0.34]
hmm1 = HMM(A,B,pi)

O = [   [0,1,1,2,0,1,2,0,0,1,2],
        [0,1,2,0,1,2],
        [0,1,2,0,0,1,2],
        [1,1,0,1,2,0,1],
        [1,2,0,0,1,2,2,0,1],
        [2,0,2,2,0,1,2,0],
        [2,0,1,2,0,1,2,0],
        [2,0,1,2,0],
        [2,0,1,2,0]
    ]
L = len(O)

# max length of O
T = len(O[0])


hmm1.BaumWelch(L,T,O, n_iter = 0)
hmm1.printhmm()
hmm1.BaumWelch(L,T,O)
hmm1.printhmm()

iteration  1
HMM content: n_states(N) = 3 ,n_features(M) = 3

hmm.pi [0.33000033 0.32890088 0.34109879]
hmm.A  [0.34033101 0.32986784 0.32980115] 	 hmm.B  [0.37057828 0.29985772 0.329564  ]
       [0.33032122 0.33979725 0.32988153] 	        [0.36039115 0.30945198 0.33015687]
       [0.33040545 0.3297981  0.33979645] 	        [0.35994557 0.29980338 0.34025105]
iteration  50
HMM content: n_states(N) = 3 ,n_features(M) = 3

hmm.pi [0.33333366 0.22096286 0.44570348]
hmm.A  [1.50006490e-01 8.49993451e-01 5.95452798e-08] 	 hmm.B  [9.99993473e-01 5.56444494e-06 9.62647769e-07]
       [7.24370624e-07 5.68017816e-02 9.43197494e-01] 	        [2.02220620e-07 9.46663454e-01 5.33363435e-02]
       [9.47426813e-01 6.83658184e-18 5.25731871e-02] 	        [1.59679766e-07 4.80269032e-02 9.51972937e-01]


### try hmmlearn

In [ ]:
from hmmlearn import hmm

x1 = [[0],[1],[1],[2],[0],[1],[2],[0],[0],[1],[2]]
x2 = [[0],[1],[2],[0],[1],[2]]
x1 = np.concatenate([x1, x2])
x2 = [[0],[1],[2],[0],[0],[1],[2]]
x1 = np.concatenate([x1, x2])
x2 = [[1],[1],[0],[1],[2],[0],[1]]
x1 = np.concatenate([x1, x2])
x2 = [[1],[2],[0],[0],[1],[2],[2],[0],[1]]
x1 = np.concatenate([x1, x2])
x2 = [[2],[0],[2],[2],[0],[1],[2],[0]]
x1 = np.concatenate([x1, x2])
x2 = [[2],[0],[1],[2],[0],[1],[2],[0]]
x1 = np.concatenate([x1, x2])
x2 = [[2],[0],[1],[2],[0]]
x1 = np.concatenate([x1, x2])
x2 = [[2],[0],[1],[2],[0]]
x1 = np.concatenate([x1, x2])

model1 = hmm.CategoricalHMM(n_components=3, n_iter=50, random_state=4)
model1.fit(x1, np.array([11,6,7,7,9,8,8,5,5]))
pi = model1.startprob_
A = model1.transmat_
B = model1.emissionprob_
print("pi\n", pi)
print("\nA\n", A)
print("\nB\n", B)

pi
 [0.33333334 0.22095794 0.44570872]

A
 [[1.50002038e-01 8.49997962e-01 3.19682997e-14]
 [9.83712805e-13 5.67642960e-02 9.43235704e-01]
 [9.47387535e-01 2.21564093e-48 5.26124650e-02]]

B
 [[9.99998038e-01 1.94951265e-06 1.23548760e-08]
 [5.35464906e-20 9.46684255e-01 5.33157450e-02]
 [6.98026863e-11 4.80422977e-02 9.51957702e-01]]


In [ ]:
print(f'A difference example implementation and hmmlearn \n{hmm1.A - (model1.transmat_)}')
print(f'\nB difference example implementation and hmmlearn \n{hmm1.B - (model1.emissionprob_)}')
print(f'\npi difference example implementation and hmmlearn \n{hmm1.pi - (model1.startprob_)}')
print(f'\nA example implementation and hmmlearn allclose: {np.allclose(hmm1.A, model1.transmat_, atol=0.1)}')
print(f'B example implementation and hmmlearn allclose: {np.allclose(hmm1.B, model1.emissionprob_, atol=0.1)}')

A difference example implementation and hmmlearn 
[[ 4.45176014e-06 -4.51130539e-06  5.95452478e-08]
 [ 7.24369640e-07  3.74855330e-05 -3.82099026e-05]
 [ 3.92779402e-05  6.83658184e-18 -3.92779402e-05]]

B difference example implementation and hmmlearn 
[[-4.56522519e-06  3.61493229e-06  9.50292893e-07]
 [ 2.02220620e-07 -2.08007056e-05  2.05984850e-05]
 [ 1.59609963e-07 -1.53944113e-05  1.52348013e-05]]

pi difference example implementation and hmmlearn 
[ 3.17152594e-07  4.92413543e-06 -5.24128803e-06]

A example implementation and hmmlearn allclose: True
B example implementation and hmmlearn allclose: True


# train set 2

In [ ]:
A = [[0.34, 0.33, 0.33], [0.33, 0.34, 0.33], [0.33, 0.33, 0.34]]
B = [[0.34, 0.33, 0.33], [0.33, 0.34, 0.33], [0.33, 0.33, 0.34]]
pi = [0.33, 0.33, 0.34]
hmm2 = HMM(A,B,pi)

O = [   [1,1,1,2,2,1,2],
        [2,2,1,0,1,1],
        [0,0,2,2,1,1,1],
        [1,1,0,1,1,0,2],
        [2,2,0,0,1,1,0,1],
        [1,1,1,2,2,1,0,0],
        [0,1,1,1,1,0,1,0],
        [2,2,2,2,2],
        [1,1,0,0,0] ]
L = len(O)

# max length of O
T = len(O[4])


hmm2.BaumWelch(L,T,O, n_iter = 0)
hmm2.printhmm()
hmm2.BaumWelch(L,T,O)
hmm2.printhmm()

iteration  1
HMM content: n_states(N) = 3 ,n_features(M) = 3

hmm.pi [0.32887801 0.33112268 0.33999931]
hmm.A  [0.33936141 0.33128305 0.32935554] 	 hmm.B  [0.26819314 0.4554281  0.27637876]
       [0.32935993 0.34131628 0.32932379] 	        [0.25876655 0.46648973 0.27474372]
       [0.32934765 0.33127328 0.33937907] 	        [0.25995845 0.45510582 0.28493573]
iteration  50
HMM content: n_states(N) = 3 ,n_features(M) = 3

hmm.pi [0.23525438 0.26316064 0.50158498]
hmm.A  [0.39422582 0.35981332 0.24596085] 	 hmm.B  [0.32442063 0.49011237 0.185467  ]
       [0.37429226 0.36776727 0.25794047] 	        [0.29345147 0.49937269 0.20717584]
       [0.27763342 0.29396266 0.42840393] 	        [0.17197341 0.38995208 0.43807451]


In [ ]:
x2 = np.array([[1],[1],[1],[2],[2],[1],[2], 
       [2],[2],[1],[0],[1],[1], 
       [0],[0],[2],[2],[1],[1],[1], 
       [1],[1],[0],[1],[1],[0],[2], 
       [2],[2],[0],[0],[1],[1],[0],[1], 
       [1],[1],[1],[2],[2],[1],[0],[0],
       [0],[1],[1],[1],[1],[0],[1],[0],
       [2],[2],[2],[2],[2],
       [1],[1],[0],[0],[0]
       ])

model2 = hmm.CategoricalHMM(n_components=3, n_iter=50)
model2.fit(x2, np.array([7,6,7,7,8,8,8,5,5]))

pi = model2.startprob_
A = model2.transmat_
B = model2.emissionprob_
print("pi\n", pi)
print("\nA\n", A)
print("\nB\n", B)

print(f'A difference example implementation and hmmlearn \n{hmm2.A - (model2.transmat_)}')
print(f'\nB difference example implementation and hmmlearn \n{hmm2.B - (model2.emissionprob_)}')
print(f'\npi difference example implementation and hmmlearn \n{hmm2.pi - (model2.startprob_)}')
print(f'\nA example implementation and hmmlearn allclose: {np.allclose(hmm2.A, model2.transmat_, atol=0.1)}')
print(f'B example implementation and hmmlearn allclose: {np.allclose(hmm2.B, model2.emissionprob_, atol=0.1)}')

pi
 [6.66666667e-01 3.33333156e-01 1.77129607e-07]

A
 [[8.68421934e-01 1.31576724e-01 1.34228662e-06]
 [1.58979450e-24 9.43461301e-04 9.99056539e-01]
 [7.14007673e-01 2.41880955e-01 4.41113718e-02]]

B
 [[3.63634071e-01 6.36365929e-01 1.66204068e-10]
 [3.20197382e-05 2.74354449e-05 9.99940545e-01]
 [1.49107958e-16 1.72828768e-09 9.99999998e-01]]
A difference example implementation and hmmlearn 
[[-0.47419611  0.2282366   0.24595951]
 [ 0.37429226  0.36682381 -0.74111607]
 [-0.43637426  0.0520817   0.38429256]]

B difference example implementation and hmmlearn 
[[-0.03921344 -0.14625356  0.185467  ]
 [ 0.29341945  0.49934525 -0.7927647 ]
 [ 0.17197341  0.38995208 -0.56192549]]

pi difference example implementation and hmmlearn 
[-0.43141229 -0.07017252  0.5015848 ]

A example implementation and hmmlearn allclose: False
B example implementation and hmmlearn allclose: False


# P1
Please specify the model parameters after the first and 50th iterations of Baum-Welch training 


## set 1

In [ ]:
A = [[0.34, 0.33, 0.33], [0.33, 0.34, 0.33], [0.33, 0.33, 0.34]]
B = [[0.34, 0.33, 0.33], [0.33, 0.34, 0.33], [0.33, 0.33, 0.34]]
pi = [0.33, 0.33, 0.34]
hmm1 = HMM(A,B,pi)

O = [ [0,1,1,2,0,1,2,0,0,1,2],
        [0,1,2,0,1,2],
        [0,1,2,0,0,1,2],
        [1,1,0,1,2,0,1],
        [1,2,0,0,1,2,2,0,1],
        [2,0,2,2,0,1,2,0],
        [2,0,1,2,0,1,2,0],
        [2,0,1,2,0],
        [2,0,1,2,0]
        ]
L = len(O)

# max length of O
T = len(O[0])

hmm1.BaumWelch(L,T,O, n_iter = 0)
hmm1.printhmm()
hmm1.BaumWelch(L,T,O)
hmm1.printhmm()

iteration  1
HMM content: n_states(N) = 3 ,n_features(M) = 3

hmm.pi [0.33000033 0.32890088 0.34109879]
hmm.A  [0.34033101 0.32986784 0.32980115] 	 hmm.B  [0.37057828 0.29985772 0.329564  ]
       [0.33032122 0.33979725 0.32988153] 	        [0.36039115 0.30945198 0.33015687]
       [0.33040545 0.3297981  0.33979645] 	        [0.35994557 0.29980338 0.34025105]
iteration  50
HMM content: n_states(N) = 3 ,n_features(M) = 3

hmm.pi [0.33333366 0.22096286 0.44570348]
hmm.A  [1.50006490e-01 8.49993451e-01 5.95452798e-08] 	 hmm.B  [9.99993473e-01 5.56444494e-06 9.62647769e-07]
       [7.24370624e-07 5.68017816e-02 9.43197494e-01] 	        [2.02220620e-07 9.46663454e-01 5.33363435e-02]
       [9.47426813e-01 6.83658184e-18 5.25731871e-02] 	        [1.59679766e-07 4.80269032e-02 9.51972937e-01]


## set 2

In [ ]:
A = [[0.34, 0.33, 0.33], [0.33, 0.34, 0.33], [0.33, 0.33, 0.34]]
B = [[0.34, 0.33, 0.33], [0.33, 0.34, 0.33], [0.33, 0.33, 0.34]]
pi = [0.33, 0.33, 0.34]
hmm2 = HMM(A,B,pi)

O = [   [1,1,1,2,2,1,2],
        [2,2,1,0,1,1],
        [0,0,2,2,1,1,1],
        [1,1,0,1,1,0,2],
        [2,2,0,0,1,1,0,1],
        [1,1,1,2,2,1,0,0],
        [0,1,1,1,1,0,1,0],
        [2,2,2,2,2],
        [1,1,0,0,0] ]
L = len(O)

# max length of O
T = len(O[4])

hmm2.BaumWelch(L,T,O, n_iter = 0)
hmm2.printhmm()
hmm2.BaumWelch(L,T,O)
hmm2.printhmm()

iteration  1
HMM content: n_states(N) = 3 ,n_features(M) = 3

hmm.pi [0.32887801 0.33112268 0.33999931]
hmm.A  [0.33936141 0.33128305 0.32935554] 	 hmm.B  [0.26819314 0.4554281  0.27637876]
       [0.32935993 0.34131628 0.32932379] 	        [0.25876655 0.46648973 0.27474372]
       [0.32934765 0.33127328 0.33937907] 	        [0.25995845 0.45510582 0.28493573]
iteration  50
HMM content: n_states(N) = 3 ,n_features(M) = 3

hmm.pi [0.23525438 0.26316064 0.50158498]
hmm.A  [0.39422582 0.35981332 0.24596085] 	 hmm.B  [0.32442063 0.49011237 0.185467  ]
       [0.37429226 0.36776727 0.25794047] 	        [0.29345147 0.49937269 0.20717584]
       [0.27763342 0.29396266 0.42840393] 	        [0.17197341 0.38995208 0.43807451]


# P2 
Please show the recognition results by using the above training sequences as the testing data (The so-called inside testing). 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
set1 = [[0,1,1,2,0,1,2,0,0,1,2],
        [0,1,2,0,1,2],
        [0,1,2,0,0,1,2],
        [1,1,0,1,2,0,1],
        [1,2,0,0,1,2,2,0,1],
        [2,0,2,2,0,1,2,0],
        [2,0,1,2,0,1,2,0],
        [2,0,1,2,0],
        [2,0,1,2,0]
        ]
set2 = [[1,1,1,2,2,1,2],
        [2,2,1,0,1,1],
        [0,0,2,2,1,1,1],
        [1,1,0,1,1,0,2],
        [2,2,0,0,1,1,0,1],
        [1,1,1,2,2,1,0,0],
        [0,1,1,1,1,0,1,0],
        [2,2,2,2,2],
        [1,1,0,0,0] ]

for i in range(len(set1)):
    leng = len(set1[i])
    print("set1[", i+1, "]", '{:>20}'.format("belongs to " + ["model 1","model 2"][hmm1.Forward(leng,set1[i]) < hmm2.Forward(leng,set1[i])]))

print()
for i in range(len(set2)):
    leng = len(set2[i])
    print("set2[", i+1, "]", '{:>20}'.format("belongs to " + ["model 1","model 2"][hmm1.Forward(leng,set2[i]) < hmm2.Forward(leng,set2[i])]))

set1[ 1 ]   belongs to model 1
set1[ 2 ]   belongs to model 1
set1[ 3 ]   belongs to model 1
set1[ 4 ]   belongs to model 1
set1[ 5 ]   belongs to model 1
set1[ 6 ]   belongs to model 1
set1[ 7 ]   belongs to model 1
set1[ 8 ]   belongs to model 1
set1[ 9 ]   belongs to model 1

set2[ 1 ]   belongs to model 2
set2[ 2 ]   belongs to model 2
set2[ 3 ]   belongs to model 2
set2[ 4 ]   belongs to model 2
set2[ 5 ]   belongs to model 2
set2[ 6 ]   belongs to model 2
set2[ 7 ]   belongs to model 2
set2[ 8 ]   belongs to model 2
set2[ 9 ]   belongs to model 2


# P3
Which class do the following testing sequences belong to?
* ABCABCCAB
* AABABCCCCBBB

In [ ]:
set3 = [[0,1,2,0,1,2,2,0,1]]
leng = len(set3[0])
print("set: ABCABCCAB", '{:>20}'.format("belongs to " + ["model 1","model 2"][hmm1.Forward(leng,set3[0]) < hmm2.Forward(leng,set3[0])]))
print()
set3 = [[1,1,2,1,2,0,0,0,0,1,1,1]]
leng = len(set3[0])
print("set: AABABCCCCBBB",  '{:>20}'.format("belongs to " + ["model 1","model 2"][hmm1.Forward(leng,set3[0]) < hmm2.Forward(leng,set3[0])]))

set: ABCABCCAB   belongs to model 1

set: AABABCCCCBBB   belongs to model 2


# P4
What are the results if Observable Markov Models were instead used in P1, P2 and P3? 

## P4-P1 set1

In [ ]:
A = [[0.34, 0.33, 0.33], [0.33, 0.34, 0.33], [0.33, 0.33, 0.34]]
B = [[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]]
pi = [0.33, 0.33, 0.34]
hmm1 = HMM(A,B,pi)

L = len(set1)

# max length of O
T = len(set1[0])

hmm1.BaumWelch(L,T,set1, n_iter = 0)
hmm1.printhmm()
hmm1.BaumWelch(L,T,set1)
hmm1.printhmm()

iteration  1
HMM content: n_states(N) = 3 ,n_features(M) = 3

hmm.pi [0.33 0.33 0.34]
hmm.A  [0.34 0.33 0.33] 	 hmm.B  [0.36367666 0.3031387  0.33318464]
       [0.33 0.34 0.33] 	        [0.36367666 0.3031387  0.33318464]
       [0.33 0.33 0.34] 	        [0.36355611 0.30281441 0.33362949]
iteration  50
HMM content: n_states(N) = 3 ,n_features(M) = 3

hmm.pi [0.18880353 0.18880353 0.62239293]
hmm.A  [0.33351497 0.32370571 0.34277932] 	 hmm.B  [0.3920173  0.36332608 0.24465662]
       [0.32370571 0.33351497 0.34277932] 	        [0.3920173  0.36332608 0.24465662]
       [0.39248806 0.39248806 0.21502389] 	        [0.30916412 0.18730313 0.50353275]


## P4-P1 set2

In [ ]:
A = [[0.34, 0.33, 0.33], [0.33, 0.34, 0.33], [0.33, 0.33, 0.34]]
B = [[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]]
pi = [0.33, 0.33, 0.34]
hmm2 = HMM(A,B,pi)

L = len(set2)

# max length of O
T = 8

hmm2.BaumWelch(L,T,set2, n_iter = 0)
hmm2.printhmm()
hmm2.BaumWelch(L,T,set2)
hmm2.printhmm()

iteration  1
HMM content: n_states(N) = 3 ,n_features(M) = 3

hmm.pi [0.33 0.33 0.34]
hmm.A  [0.34 0.33 0.33] 	 hmm.B  [0.26235652 0.4590365  0.27860698]
       [0.33 0.34 0.33] 	        [0.26235652 0.4590365  0.27860698]
       [0.33 0.33 0.34] 	        [0.26217276 0.45897636 0.27885088]
iteration  50
HMM content: n_states(N) = 3 ,n_features(M) = 3

hmm.pi [0.30598325 0.30598325 0.3880335 ]
hmm.A  [0.34595168 0.33577663 0.31827169] 	 hmm.B  [0.27297035 0.46370099 0.26332866]
       [0.33577663 0.34595168 0.31827169] 	        [0.27297035 0.46370099 0.26332866]
       [0.33026182 0.33026182 0.33947636] 	        [0.24106953 0.44970204 0.30922843]


## P4-P2 

In [ ]:
for i in range(len(set1)):
    leng = len(set1[i])
    print("set1[", i+1, "]", '{:>20}'.format("belongs to " + ["model 1","model 2"][hmm1.Forward(leng,set1[i]) < hmm2.Forward(leng,set1[i])]))

print()
for i in range(len(set2)):
    leng = len(set2[i])
    print("set2[", i+1, "]", '{:>20}'.format("belongs to " + ["model 1","model 2"][hmm1.Forward(leng,set2[i]) < hmm2.Forward(leng,set2[i])]))

set1[ 1 ]   belongs to model 1
set1[ 2 ]   belongs to model 1
set1[ 3 ]   belongs to model 1
set1[ 4 ]   belongs to model 2
set1[ 5 ]   belongs to model 1
set1[ 6 ]   belongs to model 1
set1[ 7 ]   belongs to model 1
set1[ 8 ]   belongs to model 1
set1[ 9 ]   belongs to model 1

set2[ 1 ]   belongs to model 2
set2[ 2 ]   belongs to model 2
set2[ 3 ]   belongs to model 2
set2[ 4 ]   belongs to model 2
set2[ 5 ]   belongs to model 1
set2[ 6 ]   belongs to model 2
set2[ 7 ]   belongs to model 2
set2[ 8 ]   belongs to model 1
set2[ 9 ]   belongs to model 1


## P4-P3

In [ ]:
set3 = [[0,1,2,0,1,2,2,0,1]]
leng = len(set3[0])
print("set: ABCABCCAB", '{:>20}'.format("belongs to " + ["model 1","model 2"][hmm1.Forward(leng,set3[0]) < hmm2.Forward(leng,set3[0])]))
print()
set3 = [[1,1,2,1,2,0,0,0,0,1,1,1]]
leng = len(set3[0])
print("set: AABABCCCCBBB",  '{:>20}'.format("belongs to " + ["model 1","model 2"][hmm1.Forward(leng,set3[0]) < hmm2.Forward(leng,set3[0])]))

set: ABCABCCAB   belongs to model 1

set: AABABCCCCBBB   belongs to model 2
